In [1]:
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os

In [2]:
# load our serialized face detector from disk
print("[INFO] loading face detector...")
protoPath = r'C:\Users\Om\CSD-Face-Recognition-system\opencv-face-recognition\face_detection_model\deploy.prototxt'
modelPath = r'C:\Users\Om\CSD-Face-Recognition-system\opencv-face-recognition\face_detection_model\res10_300x300_ssd_iter_140000.caffemodel'
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

[INFO] loading face detector...


In [3]:
# load our serialized face embedding model from disk
print("[INFO] loading face recognizer...")
embedder = cv2.dnn.readNetFromTorch(r'openface_nn4.small2.v1.t7')

[INFO] loading face recognizer...


In [4]:
# load the actual face recognition model along with the label encoder
recognizer = pickle.loads(open(r'recognizer_siamese.pickle', "rb").read())
reference_data = pickle.loads(open('reference_embeddings.pickle', "rb").read())
#le = pickle.loads(open(r'le.pickle', "rb").read())

Using TensorFlow backend.


In [5]:
labels=['F01','F02','F03','F04','F05']

In [6]:
# construct a blob from the image
ltestfolder=os.listdir(r'test_data')
max_proba_list=[]
label_list=[]
max_proba_index_list=[]
for images in ltestfolder:
    print(images)
    image = cv2.imread(r'test_data\\'+ images)
    image = imutils.resize(image, width=600)
    (h, w) = image.shape[:2]
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(image, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)

    # apply OpenCV's deep learning-based face detector to localize
    # faces in the input image
    detector.setInput(imageBlob)
    detections = detector.forward()
    for i in range(0, detections.shape[2]):
            # extract the confidence (i.e., probability) associated with the
            # prediction
            confidence = detections[0, 0, i, 2]

            # filter out weak detections
            if confidence > 0.3:
                # compute the (x, y)-coordinates of the bounding box for the
                # face
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                # extract the face ROI
                face = image[startY:endY, startX:endX]
                (fH, fW) = face.shape[:2]
                 
                # ensure the face width and height are sufficiently large
                if fW < 20 or fH < 20:
                    continue

                # construct a blob for the face ROI, then pass the blob
                # through our face embedding model to obtain the 128-d
                # quantification of the face
                faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96),
                    (0, 0, 0), swapRB=True, crop=False)
                embedder.setInput(faceBlob)
                vec = embedder.forward()
        
    probabilities=[]
    for reference in reference_data['embeddings']:
        reference_reshape=reference.reshape(1,1,128)
        vec_reshape=vec.reshape(1,1,128)
        preds = recognizer.predict([vec_reshape,reference_reshape])
        probabilities.append(preds)
    max_proba_index=np.argmax(probabilities)
    max_proba=probabilities[max_proba_index]
    label=labels[max_proba_index]
    max_proba_index_list.append(max_proba_index)
    max_proba_list.append(max_proba)
    label_list.append(label) 

test_image1.jpg
test_image10.jpg
test_image11.jpg
test_image12.jpg
test_image13.jpg
test_image14.jpg
test_image15.jpg
test_image16.jpg
test_image2.jpg
test_image3.jpg
test_image4.jpg
test_image5.jpg
test_image6.jpg
test_image7.jpg
test_image8.jpg
test_image9.jpg


In [7]:
print(label_list)

['F04', 'F04', 'F04', 'F05', 'F01', 'F04', 'F04', 'F02', 'F04', 'F04', 'F04', 'F03', 'F04', 'F04', 'F04', 'F04']
